# Data mining the hansard

Estimating contribution by party

## Getting debate text
- mine hansard from theyworkforyou scrape for June 2015 onwards using lxml

In [1]:
import sys
#sys.path.append('/home/fin/Documents/datamining_hansard/dh/lib/python3.4/site-packages/')

import lxml
import lxml.html
import requests
import os
import csv

In [2]:
page = requests.get("http://www.theyworkforyou.com/pwdata/scrapedxml/debates/")
tree = lxml.html.fromstring(page.text)

debates = tree.xpath('//a/text()')
debates2015 = [x for x in debates if x.startswith('debates2015')]
new_parliament = [x for x in debates2015 if int(x.split('-')[1]) > 4]

In [4]:
def download_file(url, folder):
    local_filename = url.split('/')[-1]
    if not os.path.exists(os.path.join(folder, local_filename)):
        r = requests.get(url, stream=True)
        with open(os.path.join(folder, local_filename), 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    f.flush()
        return local_filename

for xml in new_parliament:
    download_file("http://www.theyworkforyou.com/pwdata/scrapedxml/debates/"+xml,
                 "data")

## Parsing xml of debates
- XML acquired now to start parsing it using lxml

In [5]:
def get_speaker_and_text(xml):
    """
    For a given xml file return a list of tuples
    of speeches and their speakers
    """
    with open(xml, 'rb') as fh:
        parsed_xml = fh.read()
    
    data = []
    root = lxml.etree.fromstring(parsed_xml)
    speeches = root.xpath('speech') 
    for speech in speeches:
        #if speech.get('nospeaker') is not None:
        speech_list = [x.text for x in speech.getchildren() if x.text is not None]
        
        speech_list = " ".join(speech_list)
        
        speaker =  speech.get('speakername')
        data.append((speaker, speech_list))
    
    return data


## Get MP name and party affiliation

- Get a dict of MP names: party affiliation

In [6]:
with open('data/mp_data.csv',  encoding = "ISO-8859-1") as fh:
    dat = [x for x in csv.reader(fh)]

party_dict = {}
for mp in dat[1:]:
    name = mp[1] + ' ' + mp[2]
    party = mp[3]
    party_dict.update({name: party})

#add the speakers
party_dict.update({'Mr Speaker': 'Speaker'})
party_dict.update({'Mr Deputy Speaker': 'Speaker'})
party_dict.update({'Mr Speaker-Elect': 'Speaker'})
party_dict.update({'The Chairman': 'Speaker'})
party_dict.update({'The Second Deputy Chairman': 'Speaker'})

#add Stephen Philip Rotheram for some reason
party_dict.update({'Stephen Philip Rotheram': 'Labour'})

#add multiple speakers
party_dict.update({'My Lords and Members of the House of Commons':'Multiple'})
party_dict.update({'Members of the House of Commons':'Multiple'})
party_dict.update({'Hon. Members':'Multiple'})
party_dict.update({'Several hon. Members':'Multiple'})

#add none
party_dict.update({None: 'None'})

# parties and MP number
parties = [('UKIP', 1), 
           ('Sinn Féin', 4),  
           ('Conservative', 330),
           ('Labour', 232),  
           ('Liberal Democrat', 8),
           ('Independent', 1), 
           ('UUP', 2), 
           ('DUP', 8), 
           ('Scottish National Party', 56),
           ('Green', 1),
           ('Social Democratic and Labour Party', 3), 
           ('Plaid Cymru', 3)]

## Parsin' time

- Time to parse the shit out of this parliament

In [ ]:
import glob

named_text = []
for xml in glob.glob(os.path.join("data","*.xml")):
    named_text = named_text + get_speaker_and_text(xml)

In [ ]:
# get complete text for each party
party_talk = {}
for party in set(party_dict.values()):
    party_talk.update({party:''})
    
for speech in named_text:
    if speech[0] == 'Labour/Co-operative':
        speech[0] = 'Labour'
    if speech[0] in party_dict.keys():
        party = party_dict[speech[0]]
        party_talk[party] = party_talk[party] + speech[1] 
    else:
        print("Speaker not identified:", speech[0])

### Get complete text and party for each MP 
Plot of total words spoken per MP, generate a dict of MPName: (party, words spoken)

- Combine Labour + labour co-op
- Ignore None, Multiple, and Speaker

In [ ]:
words_spoken_by_mp = {}

for speaker, text in named_text:
        party = party_dict[speaker]
        # remove cruft "parties"
        if party == 'None' or party == 'Multiple' or party == 'Speaker':
            continue
        # collapse labour co-op and labour down to one party
        if party == 'Labour/Co-operative':
            party = 'Labour'
        
        if speaker in words_spoken_by_mp.keys():
            words_spoken_by_mp[speaker] = (party, words_spoken_by_mp[speaker][1] + text)
        else:
            words_spoken_by_mp.update({speaker: (party, text)})


### Get total spoken per MP

Get add party averages

In [ ]:
total_speech = []
for speaker, party_text in words_spoken_by_mp.items():
    total_speech.append((speaker, party_text[0], len(party_text[1])))

In [ ]:
for party, mps in parties:
    print(party, len(party_talk[party])/mps)